<div style="background-color: #1B1A21; text-align: right; margin-bottom: -1px">
    <img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-banner.png" style="padding: 0px; padding-right: 20px; margin: 0px; padding-top: 20px; height: 60px"/>
    <img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/banner-colors.png" style="width:100%; height: 50px; padding: 0px; margin: 0px; margin-bottom: -8px"/>
</div>

# Semantic Search with OpenAI QA

## Prerequisites for interacting with ChatGPT

### Install OpenAI package

Let's start by installing the [openai](https://platform.openai.com/docs/api-reference?lang=python) Python package.

In [5]:
!pip install openai --quiet

### Connect to ChatGPT and display the response

In [6]:
import openai

EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

You will need an OpenAI API key in order to use the the `openai` Python library.

In [9]:
openai.api_key = '<ENTER YOUR OPEN AI KEY>'

Test the connection.

In [10]:
response = openai.ChatCompletion.create(
  model=GPT_MODEL,
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the gold medal for curling in Olymics 2022?"},
    ]
)

print(response['choices'][0]['message']['content'])

I'm sorry, I cannot provide information about future events as they have not happened yet. The next Winter Olympics where curling will be contested is in 2022, but the winners have not been determined yet.


# Get the data about Winter Olympics and provide the information to ChatGPT as context

## 1. Install and import libraries

In [11]:
!pip install matplotlib --quiet
!pip install plotly.express --quiet
!pip install scikit-learn --quiet
!pip install tabulate --quiet
!pip install tiktoken --quiet
!pip install wget --quiet

In [13]:
import json
import numpy as np
import os
import pandas as pd
import wget

## 2. Fetch the CSV data and read it into a DataFrame

Download pre-chunked text and pre-computed embeddings. This file is ~200 MB, so may take a minute depending on your connection speed.

In [15]:
embeddings_url = "https://cdn.openai.com/API/examples/data/winter_olympics_2022.csv"
embeddings_path = "winter_olympics_2022.csv"

if not os.path.exists(embeddings_path):
    wget.download(embeddings_url, embeddings_path)
    print("File downloaded successfully.")
else:
    print("File already exists in the local file system.")

File downloaded successfully.


Here we are using the `converters=` parameter of the `pd.read_csv` function to convert the JSON
array in the CSV file to numpy arrays.

In [16]:
def json_to_numpy_array(x: str | None) -> np.ndarray | None:
    """Convert JSON array string into numpy array."""
    return np.array(json.loads(x)) if x else None

df = pd.read_csv(embeddings_path, converters=dict(embedding=json_to_numpy_array))
df

,text,embedding
0,Lviv bid for the 2022 Winter Olympics\n\n{{Oly...,"[-0.005021067801862955, 0.00026050032465718687..."
1,Lviv bid for the 2022 Winter Olympics\n\n==His...,"[0.0033927420154213905, -0.007447326090186834,..."
2,Lviv bid for the 2022 Winter Olympics\n\n==Ven...,"[-0.00915789045393467, -0.008366798982024193, ..."
3,Lviv bid for the 2022 Winter Olympics\n\n==Ven...,"[0.0030951891094446182, -0.006064314860850573,..."
4,Lviv bid for the 2022 Winter Olympics\n\n==Ven...,"[-0.002936174161732197, -0.006185177247971296,..."
...,...,...
6054,Anaïs Chevalier-Bouchet\n\n==Personal life==\n...,"[-0.027750400826334953, 0.001746018067933619, ..."
6055,Uliana Nigmatullina\n\n{{short description|Rus...,"[-0.021714167669415474, 0.016001321375370026, ..."
6056,Uliana Nigmatullina\n\n==Biathlon results==\n\...,"[-0.029143543913960457, 0.014654331840574741, ..."
6057,Uliana Nigmatullina\n\n==Biathlon results==\n\...,"[-0.024266039952635765, 0.011665306985378265, ..."


In [17]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059 entries, 0 to 6058
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       6059 non-null   object
 1   embedding  6059 non-null   object
dtypes: object(2)
memory usage: 94.8+ KB


## 3. Set up the database

Create the database.

In [18]:
%%sql
DROP DATABASE IF EXISTS winter_wikipedia;

CREATE DATABASE winter_wikipedia;

<div class="alert alert-block alert-danger" style="font-size: 150%; font-weight: bold">
    <p style="float: left; padding-right: 20px; padding-left: 10px"><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/caution.png"/ style="height: 55px; vertical-align: middle"></p>
    <p>Make sure to select the <tt style="font-size: 80%">winter_wikipedia</tt> database from the drop-down menu at the top of this notebook.
    It updates the <tt style="font-size: 80%">connection_url</tt> to connect to that database.</p>
</div>

In [22]:
%%sql
CREATE TABLE IF NOT EXISTS winter_olympics_2022 (
    id INT PRIMARY KEY,
    text TEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
    embedding BLOB
);

## 4. Populate the table with our DataFrame

Create a SQLAlchemy connection.

In [25]:
import sqlalchemy as sa

db_connection = sa.create_engine(connection_url).connect()

Use the `to_sql` method of the DataFrame to upload the data to the requested table.

In [26]:
df.to_sql('winter_olympics_2022', con=db_connection, index=True, index_label='id', if_exists='append', chunksize=1000)

6059

## 5. Do a semantic search with the same question from above and use the response to send to OpenAI again

In [27]:
from openai.embeddings_utils import get_embedding


def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    table_name: str,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int=100,
) -> tuple:
    """Returns a list of strings and relatednesses, sorted from most related to least."""

    # Get the embedding of the query.
    query_embedding_response = get_embedding(query, EMBEDDING_MODEL)

    # Create the SQL statement.
    stmt = f"""
        SELECT
            text,
            DOT_PRODUCT_F64(JSON_ARRAY_PACK_F64(%s), embedding) AS score
        FROM {table_name}
        ORDER BY score DESC
        LIMIT %s
    """

    # Execute the SQL statement.
    results = db_connection.execute(stmt, [json.dumps(query_embedding_response), top_n])

    strings = []
    relatednesses = []

    for row in results:
        strings.append(row[0])
        relatednesses.append(row[1])

    # Return the results.
    return strings[:top_n], relatednesses[:top_n]

In [28]:
from tabulate import tabulate

strings, relatednesses = strings_ranked_by_relatedness(
    "curling gold medal",
    df,
    "winter_olympics_2022",
    top_n=5
)

for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    print(tabulate([[string]], headers=['Result'], tablefmt='fancy_grid'))
    print('\n\n')

relatedness=0.879
╒═══════════════════════════════════════════════════╕
│ Result                                            │
╞═══════════════════════════════════════════════════╡
│ Curling at the 2022 Winter Olympics               │
│                                                   │
│ ==Medal summary==                                 │
│                                                   │
│ ===Medal table===                                 │
│                                                   │
│ {{Medals table                                    │
│  | caption        =                               │
│  | host           =                               │
│  | flag_template  = flagIOC                       │
│  | event          = 2022 Winter                   │
│  | team           =                               │
│  | gold_CAN = 0 | silver_CAN = 0 | bronze_CAN = 1 │
│  | gold_ITA = 1 | silver_ITA = 0 | bronze_ITA = 0 │
│  | gold_NOR = 0 | silver_NOR = 1 | bronze_NOR = 0 │
│  | gold_

In [30]:
import tiktoken


def num_tokens(text: str, model: str=GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from SingleStoreDB."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df, "winter_olympics_2022")
    introduction = 'Use the below articles on the 2022 Winter Olympics to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str,
    df: pd.DataFrame=df,
    model: str=GPT_MODEL,
    token_budget: int=4096 - 500,
    print_message: bool=False,
) -> str:
    """Answers a query using GPT and a table of relevant texts and embeddings in SingleStoreDB."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about the 2022 Winter Olympics."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [31]:
print(ask('Who won the gold medal for curling in Olymics 2022?'))

There were three curling events at the 2022 Winter Olympics: men's, women's, and mixed doubles. The gold medalists for each event are:

- Men's: Sweden (Niklas Edin, Oskar Eriksson, Rasmus Wranå, Christoffer Sundgren, Daniel Magnusson)
- Women's: Great Britain (Eve Muirhead, Vicky Wright, Jennifer Dodds, Hailey Duff, Mili Smith)
- Mixed doubles: Italy (Stefania Constantini, Amos Mosaner)


<img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/banner-colors-reverse.png" style="width: 100%; margin: 0; padding: 0"/>